In [1]:
import jinwu
from jinwu.core.time import Time
from jinwu.core.file import read_fits
from pathlib import Path

In [2]:
import sherpa
from sherpa.astro.io import read_table
import sherpa.io as io
from sherpa.io import read_file_data

In [3]:
lcpath = Path('/home/xinxiang/research/ep250615a/result/wxt_src.lc')

In [4]:
lc = read_fits(lcpath)

In [5]:
lc.meta.timezero

172189120.3999

In [6]:
lc

LightcurveData(path=PosixPath('/home/xinxiang/research/ep250615a/result/wxt_src.lc'), header={'XTENSION': 'BINTABLE', 'BITPIX': 8, 'NAXIS': 2, 'NAXIS1': 20, 'NAXIS2': 9, 'PCOUNT': 0, 'GCOUNT': 1, 'TFIELDS': 4, 'TTYPE1': 'TIME', 'TFORM1': 'D', 'TUNIT1': 's', 'TTYPE2': 'RATE', 'TFORM2': 'E', 'TUNIT2': 'count/s', 'TTYPE3': 'ERROR', 'TFORM3': 'E', 'TUNIT3': 'count/s', 'TTYPE4': 'FRACEXP', 'TFORM4': 'E', 'EXTNAME': 'RATE', 'HDUCLASS': 'ogip', 'HDUCLAS1': 'LIGHTCURVE', 'TIMESYS': 'TT', 'MJDREFI': 58849, 'MJDREFF': 0.00080074074, 'TIMEREF': 'LOCAL', 'TASSIGN': 'SATELLITE', 'TIMEUNIT': 's', 'TIERRELA': 1e-08, 'TIERABSO': 1.0, 'TSTART': 172189516.206, 'TSTOP': 172189566.866, 'DATE-OBS': '2025-06-15T22:26:25.3900', 'DATE-END': '2025-06-15T22:27:16.0500', 'CLOCKAPP': False, 'TELAPSE': 50.65999999642372, 'ONTIME': 50.65999999642372, 'LIVETIME': 50.65999999642372, 'EXPOSURE': 50.65999999642372, 'DEADC': 1.0, 'TIMEPIXR': 0.5, 'TIMEDEL': 5.0, 'TELESCOP': 'EP', 'INSTRUME': 'WXT', 'DETNAM': 'CMOS37', '

In [7]:
lc.time

array([400., 405., 410., 415., 420., 425., 430., 435., 440.])

In [8]:
from astropy.io import fits

In [9]:
lc.exposure


50.65999999642372

In [10]:
lc.path

PosixPath('/home/xinxiang/research/ep250615a/result/wxt_src.lc')

In [11]:
lc.header['TIMEZERO']

172189120.3999

In [12]:
lc.rate

array([9.19999981, 7.19999981, 3.20000005, 0.80000001, 1.20000005,
       0.        , 0.40000001, 0.40000001, 0.40000001])

In [13]:
lc.meta.binsize

5.0

In [14]:
lc.meta.timezero, lc.meta.tstart

(172189120.3999, 172189516.206)